**Loading Libraries And Dataset**

In [1]:
import numpy as np
import pandas as pd
import csv
import json
import statistics 
from scipy.stats import chi2
import os

In [2]:
data=pd.read_csv('../Dataset/Decadal_Air_data_states.csv')

In [3]:
data

,State,year,SO2,NO2,PM2.5
0,Andhra Pradesh,2011,6.917445,14.339150,70.653459
1,Andhra Pradesh,2012,7.021031,14.911603,72.703588
2,Andhra Pradesh,2013,7.067833,15.658970,70.840641
3,Andhra Pradesh,2014,7.587378,18.105960,73.421301
4,Andhra Pradesh,2015,7.205587,21.808934,74.846019
...,...,...,...,...,...
305,West Bengal,2016,6.975688,44.003759,83.476065
306,West Bengal,2017,6.064500,43.122708,67.246463
307,West Bengal,2018,5.153311,42.241656,51.016861
308,West Bengal,2019,4.097365,40.934686,32.767220


**Finding Expected value of All States (i.e For India) per Year**

In [4]:
expected_df=data.groupby(['year']).mean()

In [5]:
data=data.merge(expected_df,on=['year'])

**Chi Square Test For Finding Outlier**

$\chi^2=\sum_{i=1}^{i=N}\frac{(o_i-E_i)^2}{E_i}$


In [6]:
data['Chi Square Value']=((data.SO2_x-data.SO2_y)**2/data.SO2_y)+((data.NO2_x-data.NO2_y)**2/data.NO2_y)+((data['PM2.5_x']-data['PM2.5_y'])**2/data['PM2.5_y'])

**Hypothesis Test**

1) p-value obtain from Chi-Square Distribution

2) Null Hypothesis test with Significance level 1%

In [7]:
def pvalue(x):
    return 1-chi2.cdf(x,df=2)

In [8]:
data['p-value']=data['Chi Square Value'].apply(lambda x:pvalue(x))

If p-value > Significance value then it is not an outlier else it's an outlier

In [9]:
def outlier(x,significant_value):
    if x<significant_value:
        return 1
    else:
        return 0

In [10]:
data['Outlier']=data['p-value'].apply(lambda x:outlier(x,0.01))

In [11]:
data['Mpc']=(data['SO2_x']+data['NO2_x']+data['PM2.5_x'])/3

In [12]:
data=data.sort_values(['State','year'])[['State','year','Mpc','p-value','Outlier']]

In [13]:
data.columns=['State','Year','Mpc','p-value','outlier']

In [14]:
state_co=pd.read_csv('../Dataset/State-Coordinates.csv')

In [15]:
data_map=data.merge(state_co,on=['State'])

In [16]:
data.to_csv("../Dataset/Generated/chi-square/Air_Quality_chiscore.csv",index=False)

In [17]:
data_map.to_csv("../Generate Map/Datasets/chi.csv",index=False)